# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [120]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from VacationPy_api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [121]:
vacation_df = pd.read_csv("../WeatherPy/output_data/cities_weather_data.csv")
vacation_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,ramsgate,47.0,GB,1.578861e+09,93.0,51.34,1.42,46.40,11.41
1,rikitea,4.0,PF,1.578861e+09,80.0,-23.12,-134.97,76.51,17.07
2,kupang,87.0,ID,1.578861e+09,78.0,-10.17,123.58,82.85,17.72
3,hobart,20.0,AU,1.578860e+09,62.0,-42.88,147.33,63.00,12.75
4,kodiak,90.0,US,1.578861e+09,86.0,57.79,-152.41,33.80,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [122]:
# Configure gmaps
fig = gmaps.figure()

# Define variables
locations = vacation_df[["Lat", "Lng"]]
humidity = vacation_df["Humidity"]

#Create heatmap
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [123]:
# Perfect vacation variables

#Max Temp between 70 and 80 F
perfect_vacation_df = vacation_df.loc[vacation_df["Max Temp"] >= 70, :]
perfect_vacation_df = perfect_vacation_df.loc[perfect_vacation_df["Max Temp"] <= 80, :]

#Wind speed < 10 mph
perfect_vacation_df = perfect_vacation_df.loc[perfect_vacation_df["Wind Speed"] <= 10, :]

#Zero cloudiness
perfect_vacation_df = perfect_vacation_df.loc[perfect_vacation_df["Cloudiness"] == 0, :]
perfect_vacation_df.head()

#All null variables were dropped before csv used for dataframe was saved in previous activity

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
27,hermanus,0.0,ZA,1.578861e+09,82.0,-34.42,19.23,73.99,3.00
191,cidreira,0.0,BR,1.578861e+09,69.0,-30.18,-50.21,77.56,7.31
207,mbandaka,0.0,CD,1.578861e+09,65.0,0.05,18.26,74.48,2.06
236,cove,0.0,BJ,1.578861e+09,77.0,7.22,2.34,79.47,4.79
503,cururupu,0.0,BR,1.578861e+09,74.0,-1.83,-44.87,79.30,3.69


In [124]:
#Make a coordinates column to simplify API calls
perfect_vacation_df['Lat'] = perfect_vacation_df['Lat'].astype(str)
perfect_vacation_df['Lng'] = perfect_vacation_df['Lng'].astype(str)
perfect_vacation_df['Coordinates'] = perfect_vacation_df[['Lat', 'Lng']].apply(lambda x: ', '.join(x), axis=1)

# Re-order Columns
perfect_vacation_df = perfect_vacation_df[['City', 'Cloudiness', 'Country', 'Date', 'Humidity', 'Lat', 'Lng', 'Coordinates',
                                           'Max Temp', 'Wind Speed']]
perfect_vacation_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Coordinates,Max Temp,Wind Speed
27,hermanus,0.0,ZA,1.578861e+09,82.0,-34.42,19.23,"-34.42, 19.23",73.99,3.00
191,cidreira,0.0,BR,1.578861e+09,69.0,-30.18,-50.21,"-30.18, -50.21",77.56,7.31
207,mbandaka,0.0,CD,1.578861e+09,65.0,0.05,18.26,"0.05, 18.26",74.48,2.06
236,cove,0.0,BJ,1.578861e+09,77.0,7.22,2.34,"7.22, 2.34",79.47,4.79
503,cururupu,0.0,BR,1.578861e+09,74.0,-1.83,-44.87,"-1.83, -44.87",79.30,3.69
525,ampanihy,0.0,MG,1.578861e+09,62.0,-24.7,44.75,"-24.7, 44.75",79.41,6.62
543,kannur,0.0,IN,1.578861e+09,94.0,11.87,75.36,"11.87, 75.36",71.60,5.82


In [125]:
#Convert Lat and Lng back to floats
perfect_vacation_df['Lat'] = perfect_vacation_df['Lat'].astype(float)
perfect_vacation_df['Lng'] = perfect_vacation_df['Lng'].astype(float)
perfect_vacation_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Coordinates,Max Temp,Wind Speed
27,hermanus,0.0,ZA,1.578861e+09,82.0,-34.42,19.23,"-34.42, 19.23",73.99,3.00
191,cidreira,0.0,BR,1.578861e+09,69.0,-30.18,-50.21,"-30.18, -50.21",77.56,7.31
207,mbandaka,0.0,CD,1.578861e+09,65.0,0.05,18.26,"0.05, 18.26",74.48,2.06
236,cove,0.0,BJ,1.578861e+09,77.0,7.22,2.34,"7.22, 2.34",79.47,4.79
503,cururupu,0.0,BR,1.578861e+09,74.0,-1.83,-44.87,"-1.83, -44.87",79.30,3.69
525,ampanihy,0.0,MG,1.578861e+09,62.0,-24.70,44.75,"-24.7, 44.75",79.41,6.62
543,kannur,0.0,IN,1.578861e+09,94.0,11.87,75.36,"11.87, 75.36",71.60,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [126]:
#Store DataFrame in new variable
hotel_df = perfect_vacation_df

#Add "Hotel Name" to DataFrame
hotel_df["Hotel Name"] = ""

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "keyword": "hotel",
    "radius": 5000,
    "key": g_key
}

#Write loop that will iterate over dataframe
for index, row in hotel_df.iterrows():
    params['location'] = row['Coordinates']
    print(f"Retrieving Results for Index {index}: {params['location']}")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest {params['keyword']} is {results[0]['name']}")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result ... skipping")
            
    print("-"*50)

Retrieving Results for Index 27: -34.42, 19.23
Closest hotel is The Marine Hermanus
--------------------------------------------------
Retrieving Results for Index 191: -30.18, -50.21
Closest hotel is Pousada Rosa
--------------------------------------------------
Retrieving Results for Index 207: 0.05, 18.26
Closest hotel is Hotel Wimpey
--------------------------------------------------
Retrieving Results for Index 236: 7.22, 2.34
Closest hotel is Hotel La Confidence Chez Accro Miva
--------------------------------------------------
Retrieving Results for Index 503: -1.83, -44.87
Closest hotel is Hotel Líder
--------------------------------------------------
Retrieving Results for Index 525: -24.7, 44.75
Missing field/result ... skipping
--------------------------------------------------
Retrieving Results for Index 543: 11.87, 75.36
Closest hotel is Mascot Beach Resort
--------------------------------------------------


In [127]:
# Drop rows with null values
hotel_df.replace(r'^\s*$', np.nan, regex=True, inplace = True)
hotel_df = hotel_df.dropna(axis = 0, how ='any')

# Preview DataFrame
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Coordinates,Max Temp,Wind Speed,Hotel Name
27,hermanus,0.0,ZA,1.578861e+09,82.0,-34.42,19.23,"-34.42, 19.23",73.99,3.00,The Marine Hermanus
191,cidreira,0.0,BR,1.578861e+09,69.0,-30.18,-50.21,"-30.18, -50.21",77.56,7.31,Pousada Rosa
207,mbandaka,0.0,CD,1.578861e+09,65.0,0.05,18.26,"0.05, 18.26",74.48,2.06,Hotel Wimpey
236,cove,0.0,BJ,1.578861e+09,77.0,7.22,2.34,"7.22, 2.34",79.47,4.79,Hotel La Confidence Chez Accro Miva
503,cururupu,0.0,BR,1.578861e+09,74.0,-1.83,-44.87,"-1.83, -44.87",79.30,3.69,Hotel Líder
543,kannur,0.0,IN,1.578861e+09,94.0,11.87,75.36,"11.87, 75.36",71.60,5.82,Mascot Beach Resort


In [128]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [129]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)

#Display map
fig

Figure(layout=FigureLayout(height='420px'))